In [ ]:
# Step 1: Install dependencies (Run only once)
!pip install prophet pandas matplotlib openpyxl



In [ ]:
# Step 2: Import libraries
import pandas as pd
from prophet import Prophet
from google.colab import files
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
# Step 3: Upload and Load Data File
uploaded = files.upload()  # Opens a file upload dialog
file_name = list(uploaded.keys())[0]  # Gets the uploaded filename
if file_name.endswith('.csv'):
    df = pd.read_csv(file_name)
elif file_name.endswith('.xls') or file_name.endswith('.xlsx'):
    df = pd.read_excel(file_name)
else:
    print("Unsupported file format. Please upload a .csv or .xlsx file.")
    raise ValueError("Unsupported file format")

print("Columns in the DataFrame:")
print(df.columns)



Saving Raw_data.xlsx to Raw_data.xlsx
Columns in the DataFrame:
Index(['ID', 'Parent_Company', 'Acquisition_Year', 'Acquisition_Month',
       'Acquired_Company', 'Business', 'Country', 'Acquisition_Price',
       'Category', 'Derived Products'],
      dtype='object')


In [ ]:
# Step 4: Data Cleaning
df['Acquisition_Year'] = pd.to_numeric(df['Acquisition_Year'], errors='coerce')
df = df.dropna(subset=['Acquisition_Year'])
df['Parent_Company'] = df['Parent_Company'].fillna('Unknown')

# Step 5: Interactive Year-wise Acquisition Count by Company
companies = df['Parent_Company'].unique()
fig = go.Figure()
for i, company in enumerate(companies):
    temp_df = df[df['Parent_Company'] == company]
    yearly_counts = temp_df.groupby('Acquisition_Year').size().reset_index(name='Acquisition_Count')
    fig.add_trace(go.Bar(x=yearly_counts['Acquisition_Year'], y=yearly_counts['Acquisition_Count'],
                         name=f"{company} (Bar)", visible=(i == 0), marker_color='skyblue'))
    fig.add_trace(go.Scatter(x=yearly_counts['Acquisition_Year'], y=yearly_counts['Acquisition_Count'],
                             name=f"{company} (Line)", mode='lines+markers', visible=(i == 0), line=dict(color='blue', width=3)))
dropdown_buttons = []
for i, company in enumerate(companies):
    visibility = [False] * len(companies) * 2
    visibility[2*i] = True
    visibility[2*i+1] = True
    dropdown_buttons.append(dict(label=company, method='update', args=[{'visible': visibility}, {'title': f'Year-wise Acquisition Count – {company}'}]))
fig.update_layout(
    updatemenus=[dict(active=0, buttons=dropdown_buttons, x=0.01, y=1.15, showactive=True)],
    title='Year-wise Acquisition Count – ' + companies[0],
    xaxis_title='Year',
    yaxis_title='Number of Acquisitions',
    template='plotly_white',
    hovermode='x unified',
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)
fig.show()

In [ ]:
# Clean price column
df['Acquisition_Price'] = pd.to_numeric(df['Acquisition_Price'], errors='coerce')
df = df.dropna(subset=['Acquisition_Year', 'Parent_Company'])
# Only keep rows with a price
df_price = df.dropna(subset=['Acquisition_Price'])

# Outliers
def get_outliers(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return (series < lower) | (series > upper)
# Prepare traces for each company
companies = df_price['Parent_Company'].unique()
data = []
buttons = []
for i, company in enumerate(companies):
    temp = df_price[df_price['Parent_Company'] == company]
    outliers = get_outliers(temp['Acquisition_Price'])
    # All points
    scatter_all = go.Scatter(
        x=temp['Acquisition_Year'],
        y=temp['Acquisition_Price'],
        mode='markers',
        marker=dict(color='blue', size=8),
        name='All Prices',
        visible=(i == 0)
    )
    # Outliers
    scatter_out = go.Scatter(
        x=temp['Acquisition_Year'][outliers],
        y=temp['Acquisition_Price'][outliers],
        mode='markers',
        marker=dict(color='red', size=12, symbol='diamond'),
        name='Outliers',
        visible=(i == 0)
    )
    data.extend([scatter_all, scatter_out])
    # Button for dropdown
    vis = [False] * len(companies) * 2
    vis[2*i] = vis[2*i+1] = True
    buttons.append(dict(label=company,
                       method='update',
                       args=[{'visible': vis},
                             {'title': f'Acquisition Price Outliers – {company}'}]))
# Create figure
fig = go.Figure(data=data)
fig.update_layout(
    updatemenus=[dict(type='dropdown', buttons=buttons, x=0.01, y=1.15)],
    title=f'Acquisition Price Outliers – {companies[0]}',
    xaxis_title='Year',
    yaxis_title='Acquisition Price (USD)',
    template='plotly_white'
)
fig.show()

In [ ]:
# Heatmap
pivot = pd.pivot_table(
    df,
    index='Parent_Company',
    columns='Acquisition_Year',
    values='Acquired_Company',
    aggfunc='count',
    fill_value=0
)
# Convert to binary: 1 (acquisition), 0 (no acquisition)
pivot = (pivot > 0).astype(int)
# Interactive heatmap
fig = px.imshow(
    pivot,
    aspect='auto',
    color_continuous_scale=['#e0e0e0', '#003366'],  # gray for missing, blue for acquisitions
    labels=dict(x="Year", y="Company", color="Acquisition"),
    title="Acquisition Occurrence Heatmap (Blue = Acquisition, Gray = No Data)"
)
fig.update_xaxes(side="top")
fig.show()

In [ ]:

# Function: Company-Year wise outlier flag
def flag_outliers(group):
    Q1 = group['Acquisition_Price'].quantile(0.25)
    Q3 = group['Acquisition_Price'].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    group['Is_Outlier'] = (group['Acquisition_Price'] < lower) | (group['Acquisition_Price'] > upper)
    return group
# Company-Year wise apply
df_flagged = df_price.groupby(['Parent_Company', 'Acquisition_Year'], group_keys=False).apply(flag_outliers)
# Company list
companies = df_flagged['Parent_Company'].unique()
# Interactive boxplot traces
data = []
buttons = []
for i, company in enumerate(companies):
    temp = df_flagged[df_flagged['Parent_Company'] == company]
    # Before removal
    box_all = go.Box(
        y=temp['Acquisition_Price'],
        x=temp['Acquisition_Year'],
        name='Original',
        marker_color='blue',
        boxmean=True,
        boxpoints='outliers',
        visible=(i==0)
    )
    # After removal
    box_no = go.Box(
        y=temp.loc[~temp['Is_Outlier'], 'Acquisition_Price'],
        x=temp.loc[~temp['Is_Outlier'], 'Acquisition_Year'],
        name='Without Outliers',
        marker_color='green',
        boxmean=True,
        boxpoints='outliers',
        visible=(i==0)
    )
    data.extend([box_all, box_no])
    # Dropdown button
    vis = [False]*len(companies)*2
    vis[2*i] = vis[2*i+1] = True
    buttons.append(dict(label=company,
                       method='update',
                       args=[{'visible': vis},
                             {'title': f'Company: {company} - Acquisition Price Boxplot (Year-wise)'}]))
# Figure
fig = go.Figure(data=data)
fig.update_layout(
    updatemenus=[dict(type='dropdown', buttons=buttons, x=0.01, y=1.15)],
    title=f'Company: {companies[0]} - Acquisition Price Boxplot (Year-wise)',
    xaxis_title='Acquisition Year',
    yaxis_title='Acquisition Price (USD)',
    template='plotly_white'
)
fig.show()

<ipython-input-19-36a9814ec7ac>:11: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [ ]:

# Step 6: Top 10 Companies by Total Acquisitions
top10 = (
    df['Parent_Company']
    .value_counts()
    .nlargest(10)
    .reset_index()
    .rename(columns={'index': 'Parent_Company', 'count': 'Acquisition_Count'})
)
fig = px.bar(
    top10,
    x='Acquisition_Count',
    y='Parent_Company',
    orientation='h',
    title=':trophy: Top 10 Companies with Most Acquisitions',
    text='Acquisition_Count',
    labels={'Parent_Company': 'Company', 'Acquisition_Count': 'Number of Acquisitions'},
    color='Acquisition_Count',
    color_continuous_scale='Blues',
    template='plotly_white'
)
fig.update_traces(textposition='outside')
fig.update_layout(yaxis=dict(autorange='reversed'), font=dict(size=14), title_font_size=24)
fig.show()


In [ ]:

# Step 7: Top 5 Most Expensive Acquisitions
df['Acquisition_Price'] = pd.to_numeric(df['Acquisition_Price'], errors='coerce')
df_filtered = df.dropna(subset=['Acquisition_Price'])
if not df_filtered.empty:
    top_5_expensive = df_filtered.sort_values(by='Acquisition_Price', ascending=False).head(5)
    fig = px.bar(top_5_expensive, x='Acquired_Company', y='Acquisition_Price',
                 title='Top 5 Most Expensive Acquisitions (Overall)',
                 labels={'Acquired_Company': 'Acquired Company', 'Acquisition_Price': 'Acquisition Price (USD)'},
                 color='Acquisition_Price', color_continuous_scale='viridis',
                 hover_data=['Acquisition_Year', 'Parent_Company'], template='plotly_white')
    fig.update_traces(text=top_5_expensive['Acquisition_Price'].apply(lambda x: f"${x/1e9:.2f}B"), textposition='outside')
    fig.update_layout(xaxis_title='Acquired Company', yaxis_title='Acquisition Price (USD)', xaxis=dict(tickangle=-45, automargin=True))
    fig.show()
else:
    print("No acquisition price data available to plot the top 5 most expensive acquisitions.")



In [ ]:
# Step 8: Average Acquisition Price by Month
df_filtered = df.dropna(subset=['Acquisition_Price', 'Acquisition_Month'])
if not df_filtered.empty:
    average_price_by_month = df_filtered.groupby('Acquisition_Month')['Acquisition_Price'].mean().reset_index()
    month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    average_price_by_month['Acquisition_Month'] = pd.Categorical(average_price_by_month['Acquisition_Month'], categories=month_order, ordered=True)
    average_price_by_month = average_price_by_month.sort_values('Acquisition_Month')
    fig = px.bar(average_price_by_month, x='Acquisition_Month', y='Acquisition_Price',
                 title='Average Acquisition Price by Month',
                 labels={'Acquisition_Month': 'Month', 'Acquisition_Price': 'Average Price (in Billions USD)'},
                 template='plotly_white')
    fig.update_yaxes(tickprefix='$', tickformat=".2s", title='Average Price (Billions USD)')
    fig.update_xaxes(title='Month of Acquisition')
    fig.show()
else:
    print("No acquisition price or month data available to plot price by month.")

In [ ]:

# Step 9: Filter Eligible Companies for Forecasting
company_stats = df.groupby('Parent_Company').agg(
    total_acquisitions=('Acquired_Company', 'count'),
    unique_years=('Acquisition_Year', pd.Series.nunique),
    non_null_prices=('Acquisition_Price', lambda x: x.notnull().sum())
).reset_index()
eligible_companies = company_stats[
    (company_stats['total_acquisitions'] >= 3) &
    (company_stats['unique_years'] >= 2) &
    (company_stats['non_null_prices'] >= 2)
]['Parent_Company'].tolist()
print(":white_check_mark: Eligible Companies:", eligible_companies)


:white_check_mark: Eligible Companies: ['Adobe', 'Amazon', 'Apple', 'Blackberry', 'Disney', 'Ebay', 'Facebook', 'Google', 'Hp', 'Ibm', 'Microsoft', 'Redhat', 'Twitter']


In [ ]:

# Step 10: Forecasting Function (with Evaluation)
def forecast_with_eval(df_subset, col_name, company_name, forecast_years=5, test_years=3, eval_results_list=None):
    df_subset = df_subset.dropna(subset=['Acquisition_Year']).copy()
    df_prophet = df_subset[['Acquisition_Year', col_name]].copy()
    df_prophet.rename(columns={'Acquisition_Year': 'ds', col_name: 'y'}, inplace=True)
    df_prophet['ds'] = pd.to_datetime(df_prophet['ds'].astype(int), format='%Y')
    df_prophet['year'] = df_prophet['ds'].dt.year
    df_prophet = df_prophet.sort_values('ds')
    if len(df_prophet) <= test_years + 2:
        print(f":warning: Not enough data for {company_name} | {col_name} to perform train/test split.")
        if eval_results_list is not None:
            eval_results_list.append({'Parent_Company': company_name, 'Metric': col_name, 'MAE': 'N/A', 'RMSE': 'N/A', 'MAPE': 'N/A'})
        return pd.DataFrame()
    train_df = df_prophet.iloc[:-test_years].copy()
    test_df = df_prophet.iloc[-test_years:].copy()
    model = Prophet(yearly_seasonality=True)
    model.fit(train_df[['ds', 'y']])
    future = model.make_future_dataframe(periods=forecast_years, freq='Y')
    forecast = model.predict(future)
    forecast['year'] = forecast['ds'].dt.year
    forecast = forecast[['ds', 'year', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
    forecast[['yhat', 'yhat_lower', 'yhat_upper']] = forecast[['yhat', 'yhat_lower', 'yhat_upper']].clip(lower=0)
    if col_name == 'Acquisition_Count':
        forecast[['yhat', 'yhat_lower', 'yhat_upper']] = forecast[['yhat', 'yhat_lower', 'yhat_upper']].round().astype(int)
    merged_eval = pd.merge(test_df[['year', 'y']], forecast[['year', 'yhat']], on='year', how='inner')
    if merged_eval.empty or merged_eval['y'].isnull().all():
        if eval_results_list is not None:
            eval_results_list.append({'Parent_Company': company_name, 'Metric': col_name, 'MAE': 'N/A', 'RMSE': 'N/A', 'MAPE': 'N/A'})
        df_combined = pd.merge(df_prophet[['year', 'y']].rename(columns={'y': 'Actual'}), forecast[['year', 'yhat', 'yhat_lower', 'yhat_upper']], on='year', how='outer')
        df_combined.insert(0, 'Parent_Company', company_name)
        df_combined.insert(1, 'Metric', col_name)
        return df_combined[['Parent_Company', 'Metric', 'year', 'Actual', 'yhat', 'yhat_lower', 'yhat_upper']]
    mae = mean_absolute_error(merged_eval['y'], merged_eval['yhat'])
    rmse = np.sqrt(mean_squared_error(merged_eval['y'], merged_eval['yhat']))
    mape = np.mean(np.abs((merged_eval['y'] - merged_eval['yhat']) / merged_eval['y'].replace(0, np.nan))) * 100
    mape_str = f"{mape:.2f}%" if not (np.isnan(mape) or np.isinf(mape)) else "N/A"
    print(f":bar_chart: {company_name} | {col_name} | MAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape_str}")
    if eval_results_list is not None:
        eval_results_list.append({'Parent_Company': company_name, 'Metric': col_name, 'MAE': f"{mae:.2f}", 'RMSE': f"{rmse:.2f}", 'MAPE': mape_str})
    df_combined = pd.merge(df_prophet[['year', 'y']].rename(columns={'y': 'Actual'}), forecast[['year', 'yhat', 'yhat_lower', 'yhat_upper']], on='year', how='outer')
    df_combined.insert(0, 'Parent_Company', company_name)
    df_combined.insert(1, 'Metric', col_name)
    return df_combined[['Parent_Company', 'Metric', 'year', 'Actual', 'yhat', 'yhat_lower', 'yhat_upper']]


In [ ]:

# Step 11: Run Forecasts for All Eligible Companies
final_output = []
evaluation_results = []
for company in eligible_companies:
    print(f"\n:crystal_ball: Forecasting for: {company}")
    count_df = df[df['Parent_Company'] == company].groupby('Acquisition_Year').size().reset_index(name='Acquisition_Count')
    count_df['Parent_Company'] = company
    forecast_count = forecast_with_eval(count_df, 'Acquisition_Count', company, eval_results_list=evaluation_results)
    if not forecast_count.empty:
        final_output.append(forecast_count)
    price_df = df[df['Parent_Company'] == company].groupby('Acquisition_Year')['Acquisition_Price'].sum().reset_index()
    price_df['Parent_Company'] = company
    if price_df['Acquisition_Price'].notnull().sum() >= 2:
        forecast_price = forecast_with_eval(price_df, 'Acquisition_Price', company, eval_results_list=evaluation_results)
        if not forecast_price.empty:
            final_output.append(forecast_price)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 17.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpcpirqx7t/d_zk4qt5.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpcpirqx7t/9ii62g7h.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=69456', 'data', 'file=/tmp/tmpcpirqx7t/d_zk4qt5.json', 'init=/tmp/tmpcpirqx7t/9ii62g7h.json', 'output', 'file=/tmp/tmpcpirqx7t/prophet_model8b0jei54/prophet_model-20250522081610.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
08:16:10 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing



:crystal_ball: Forecasting for: Adobe


08:16:10 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1854: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 17.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpcpirqx7t/2m5mqt7h.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpcpirqx7t/aywsd2dt.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=55658', 'data', 'file=/tmp/tmpcpirqx7t/2m5mqt7h.json', 'init=/tmp/tmpcpirqx7t/aywsd2dt.json', 'output', 'file=/tmp/tmpcpirqx7t/prop

:bar_chart: Adobe | Acquisition_Count | MAE: 1.00, RMSE: 1.29, MAPE: 50.00%
:bar_chart: Adobe | Acquisition_Price | MAE: 2641854565.24, RMSE: 3811456885.41, MAPE: 99.85%

:crystal_ball: Forecasting for: Amazon


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 13.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpcpirqx7t/rw01vjjc.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpcpirqx7t/s80s7sdm.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=54343', 'data', 'file=/tmp/tmpcpirqx7t/rw01vjjc.json', 'init=/tmp/tmpcpirqx7t/s80s7sdm.json', 'output', 'file=/tmp/tmpcpirqx7t/prophet_model1v8kn178/prophet_model-20250522081611.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
08:16:11 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
08:16:11 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1854

...

08:16:14 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1854: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 11.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpcpirqx7t/_nx0lvwd.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpcpirqx7t/oa4n5aq6.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=15986', 'data', 'file=/tmp/tmpcpirqx7t/_nx0lvwd.json', 'init=/tmp/tmpcpirqx7t/oa4n5aq6.json', 'output', 'file=/tmp/tmpcpirqx7t/prop

:bar_chart: Redhat | Acquisition_Count | MAE: 0.50, RMSE: 0.71, MAPE: 25.00%
:bar_chart: Redhat | Acquisition_Price | MAE: 108188256.29, RMSE: 119556461.51, MAPE: 22.92%

:crystal_ball: Forecasting for: Twitter


08:16:14 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1854: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpcpirqx7t/917rcd5c.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpcpirqx7t/zqxre626.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=53401', 'data', 'file=/tmp/tmpcpirqx7t/917rcd5c.json', 'init=/tmp/tmpcpirqx7t/zqxre626.json', 'output', 'file=/tmp/tmpcpirqx7t/proph

:bar_chart: Twitter | Acquisition_Count | MAE: 3.67, RMSE: 3.70, MAPE: 113.89%
:bar_chart: Twitter | Acquisition_Price | MAE: 21408765906.39, RMSE: 21437936553.87, MAPE: N/A


/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1854: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.



In [ ]:
# Step 12: Display Evaluation Metrics
if evaluation_results:
    eval_df = pd.DataFrame(evaluation_results)
    print("\n:bar_chart: Forecasting Model Evaluation Metrics:")
    display(eval_df)
else:
    print("\n:no_entry_sign: No evaluation metrics could be calculated.")



In [ ]:
# Step 13: Plot Forecasts for Each Company (Interactive Subplots)
if final_output:
    all_forecast_data = pd.concat(final_output, ignore_index=True)
    forecast_companies = all_forecast_data['Parent_Company'].dropna().unique()
    fig = make_subplots(rows=2, cols=1, subplot_titles=('Acquisition Count Forecast', 'Acquisition Price Forecast'), shared_xaxes=True, vertical_spacing=0.1)
    for i, company in enumerate(forecast_companies):
        company_data = all_forecast_data[all_forecast_data['Parent_Company'] == company]
        count_data = company_data[company_data['Metric'] == 'Acquisition_Count']
        price_data = company_data[company_data['Metric'] == 'Acquisition_Price']
        fig.add_trace(go.Scatter(x=count_data['year'], y=count_data['Actual'], mode='lines+markers', name=f'{company} - Count (Actual)', line=dict(color='blue'), visible=(i == 0)), row=1, col=1)
        fig.add_trace(go.Scatter(x=count_data['year'], y=count_data['yhat'], mode='lines', name=f'{company} - Count (Forecast)', line=dict(color='red', dash='dash'), visible=(i == 0)), row=1, col=1)
        fig.add_trace(go.Scatter(x=count_data['year'], y=count_data['yhat_upper'], mode='lines', line=dict(width=0), hoverinfo='skip', showlegend=False, visible=(i == 0)), row=1, col=1)
        fig.add_trace(go.Scatter(x=count_data['year'], y=count_data['yhat_lower'], mode='lines', line=dict(width=0), fill='tonexty', fillcolor='rgba(255,0,0,0.2)', hoverinfo='skip', showlegend=False, visible=(i == 0)), row=1, col=1)
        fig.add_trace(go.Scatter(x=price_data['year'], y=price_data['Actual'], mode='lines+markers', name=f'{company} - Price (Actual)', line=dict(color='green'), visible=(i == 0)), row=2, col=1)
        fig.add_trace(go.Scatter(x=price_data['year'], y=price_data['yhat'], mode='lines', name=f'{company} - Price (Forecast)', line=dict(color='purple', dash='dash'), visible=(i == 0)), row=2, col=1)
        fig.add_trace(go.Scatter(x=price_data['year'], y=price_data['yhat_upper'], mode='lines', line=dict(width=0), hoverinfo='skip', showlegend=False, visible=(i == 0)), row=2, col=1)
        fig.add_trace(go.Scatter(x=price_data['year'], y=price_data['yhat_lower'], mode='lines', line=dict(width=0), fill='tonexty', fillcolor='rgba(128,0,128,0.2)', hoverinfo='skip', showlegend=False, visible=(i == 0)), row=2, col=1)
    dropdown_buttons = []
    num_traces_per_company = 8
    for i, company in enumerate(forecast_companies):
        visibility = [False] * (len(forecast_companies) * num_traces_per_company)
        idx = i * num_traces_per_company
        for j in range(num_traces_per_company):
            visibility[idx + j] = True
        dropdown_buttons.append(dict(label=company, method='update', args=[{'visible': visibility}, {'title': f'Acquisition Forecasts - {company}'}]))
    fig.update_layout(
        updatemenus=[dict(active=0, buttons=dropdown_buttons, x=0.01, y=1.15, showactive=True)],
        title=f'Acquisition Forecasts - {forecast_companies[0]}',
        yaxis1=dict(title='Number of Acquisitions'),
        yaxis2=dict(title='Total Acquisition Price (USD)'),
        xaxis=dict(title='Year'),
        template='plotly_white',
        hovermode='x unified',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    fig.show()
else:
    print("No eligible companies or sufficient data to generate plots.")

In [ ]:
# Combine forecast outputs
final_df = pd.concat(final_output, ignore_index=True)

# Convert evaluation results to DataFrame
eval_df = pd.DataFrame(evaluation_results)

# Save to Excel
with pd.ExcelWriter("Forecast_Results.xlsx", engine='openpyxl') as writer:
    final_df.to_excel(writer, sheet_name='Forecasts', index=False)
    eval_df.to_excel(writer, sheet_name='Evaluation_Metrics', index=False)

# Download the file
files.download("Forecast_Results.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>